In [2]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [9]:
def read_text_data(fname):
    txtdata = []
    classes = []
    topics  = []
    with open(fname, 'r', encoding='UTF-', errors='ignore') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            if len(row)>1:
                # get the text
                txtdata.append(row[6])
                # get the class (convert to integer)
                if len(row)>1:
                    classes.append(row[1])
    
    if (len(classes)>0) and (len(txtdata) != len(classes)):        
        raise Exception("mismatched length!"+ str(len(txtdata))+'***'+str(len(classes)))
    
    return (txtdata, classes)

def read_text_data1(fname):
    txtdata = []
    classes = []
    topics  = []
    with open(fname, 'r', encoding='UTF-', errors='ignore') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            if len(row)>1:
                # get the text
                txtdata.append(row[0])
                # get the class (convert to integer)
                if len(row)>1:
                    classes.append(row[1])
    
    if (len(classes)>0) and (len(txtdata) != len(classes)):        
        raise Exception("mismatched length!"+ str(len(txtdata))+'***'+str(len(classes)))
    
    return (txtdata, classes)


def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [4]:
# load the data
# (if using Kaggle notebooks you need to include the directory path: /kaggle/input/cs5489-2020b-assignment-1/)
# (traintxt, trainY, traintopic) = read_text_data("sanders_tweets_train.txt")
(traintxt, trainY) = read_text_data("train.csv")
(testtxt, _)                = read_text_data1("Olympics_Tokyo_tweets-copy1.csv")
print(len(traintxt))
print(len(testtxt))

5001
311439


In [5]:
# INSERT YOUR CODE HERE
cntvect = feature_extraction.text.CountVectorizer(stop_words='english', max_features=90000)
cntvect.fit(traintxt)
# calculate the vectors for the training data
trainX = cntvect.transform(traintxt)
# calculate vectors for the test data
testX = cntvect.transform(testtxt)
# print the vocabulary
# - (key,value) pairs correspond to (word,vector index)

In [6]:
# TF-IDF representation
# (For TF, pass use_idf=False)
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1', smooth_idf = False)
# setup the TF-IDF representation, and transform the training set
trainXtf = tf_trans.fit_transform(trainX)
# transform the test set
testXtf = tf_trans.transform(testX)

In [7]:
#svm
# setup the list of parameters to try
paramgrid = {'C': logspace(-2,3,20),
            'gamma': logspace(-4,3,20),
            }
# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv = model_selection.GridSearchCV(svm.SVC(kernel='rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)
# run cross-validation (train for each split)
svmcv.fit(trainXtf, trainY);
best_param = svmcv.best_params_
clf = svm.SVC(kernel='rbf',C=best_param['C'],gamma=best_param['gamma'])
clf.fit(trainXtf,trainY)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


SVC(C=26.366508987303583, gamma=2.6366508987303554)

In [31]:
import time
start = time.time()


predYnew = clf.predict(testXtf)
write_csv_kaggle_sub("predictall.csv", predYnew)

end = time.time()
print( end - start)

75.2939441204071


In [39]:
# shape(trainXtf)[0]
# print('{} : {}'.format(n_cores, (end - start)))

1 : 75.2939441204071


In [8]:
from joblib import Parallel, delayed
# from sklearn import svm

# data_train = [[0,2,3],[1,2,3],[4,2,3]]
# targets_train = [0,1,0]

# clf = svm.SVC(kernel='rbf', degree=3, C=10, gamma=0.3, probability=True)
# clf.fit(data_train, targets_train)

# to_be_predicted = np.array([[1,3,4], [1,3,4], [1,3,5]])
# clf.predict(to_be_predicted)
#看看不同workers用多久
for n_cores in range(1,17):
# n_cores = 8

    parallel = Parallel(n_jobs=n_cores)

    start1 = time.time()


    results = parallel(delayed(clf.predict)(testXtf[i].reshape(-1,12776))
    for i in range(shape(testXtf)[0]))

    predYnew = vstack(results).flatten()
    write_csv_kaggle_sub("predictall_parallel.csv", predYnew)

    end1 = time.time()
    print('{} : {}'.format(n_cores, (end1 - start1)))

NameError: name 'time' is not defined

In [12]:
#分天预测
from joblib import Parallel, delayed
clf = svm.SVC(kernel = 'rbf',C=26.366508987303583, gamma=2.6366508987303554)
clf.fit(trainXtf, trainY)
timelist = ['2021-07-24','2021-07-25','2021-07-26','2021-07-27','2021-07-28','2021-07-29','2021-07-30','2021-07-31','2021-08-07','2021-08-08']

for a in timelist:
    (testtxt, _)                = read_text_data(a + ".csv")
    testX = cntvect.transform(testtxt)
    testXtf = tf_trans.transform(testX)

    n_cores = 7

    parallel = Parallel(n_jobs=n_cores)

    # start1 = time.time()


    results = parallel(delayed(clf.predict)(testXtf[i].reshape(-1,12776))
    for i in range(shape(testXtf)[0]))

    predYnew = vstack(results).flatten()
    write_csv_kaggle_sub("predictall_parallel" + a + ".csv", predYnew)

# end1 = time.time()
# print('{} : {}'.format(n_cores, (end1 - start1)))